In [ ]:
import yaml
import ultralytics
from ultralytics import YOLO
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import shutil
import random

In [ ]:
# 데이터 설정
data = {
    'train': '/train_data/train/images',
    'val': '/train_data/val/images',
    'test': '/train_data/test/images',
    'nc': 5,
    'names': ['idcard', 'arrow', 'hanoi', 'people', 'pan']
}

# yaml 파일 생성
yaml_file = './ultralytics/train_data/data.yaml'  # 저장할 위치와 파일명 지정

with open(yaml_file, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print(f"YAML file saved at: {yaml_file}")


In [ ]:
#모델 학습
ultralytics.checks()

model = YOLO('yolov8n.pt')
print(type(model.names), len(model.names))

print(model.names)

model.train(data='./ultralytics/train_data/data.yaml', epochs=100)

In [ ]:
# 모델 평가

# CSV 파일 로드
file_path = './ultralytics/runs/detect/train/results.csv'
data = pd.read_csv(file_path)

# 열 이름의 앞뒤 공백 제거
data.columns = data.columns.str.strip()

# mAP@50과 mAP@50-95 데이터 추출
mAP50 = data['metrics/mAP50(B)']
mAP50_95 = data['metrics/mAP50-95(B)']
epochs = data['epoch']

# 그래프 그리기
plt.figure(figsize=(10, 6))
plt.plot(epochs, mAP50, label='mAP@50', marker='o')
plt.plot(epochs, mAP50_95, label='mAP@50-95', marker='o')
plt.xlabel('Epochs')
plt.ylabel('mAP')
plt.title('YOLO Model mAP Analysis Over Epochs')
plt.legend()
plt.grid(True)

# 그래프를 파일로 저장
plt.savefig('output_graph.png')


In [ ]:
# 데이터 증강

input_image_dir = "/yolov8/ultralytics/train_data1/train/images"
input_label_dir = "/yolov8/ultralytics/train_data1/train/labels"
output_image_dir = "/yolov8/ultralytics/train_data3/train/images"
output_label_dir = "/yolov8/ultralytics/train_data3/train/labels"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# 밝기 조정
brightness_factors = np.arange(0.5, 1.4, 0.05)

for filename in os.listdir(input_image_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
  
        image_path = os.path.join(input_image_dir, filename)
        image = cv2.imread(image_path)

        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(input_label_dir, label_filename)

        # 라벨 파일 복사
        for factor in brightness_factors:
            # 밝기 조정
            augmented_image = cv2.convertScaleAbs(image, alpha=factor, beta=0)

            output_image_filename = f"bright_{factor:.2f}_{filename}"
            output_image_path = os.path.join(output_image_dir, output_image_filename)
            cv2.imwrite(output_image_path, augmented_image)

           
            output_label_filename = f"bright_{factor:.2f}_{label_filename}"
            output_label_path = os.path.join(output_label_dir, output_label_filename)
            shutil.copy(label_path, output_label_path)

        print(f"{filename}의 밝기 조정 및 라벨 파일 복사 완료")

In [ ]:
# 데이터 증강(라벨 데이터)
input_image_dir = "/yolov8/ultralytics/train_data1/val/images"
input_label_dir = "/yolov8/ultralytics/train_data1/val/labels"
output_image_dir = "/yolov8/ultralytics/train_data3/val/images"
output_label_dir = "/yolov8/ultralytics/train_data3/val/labels"

os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# 밝기 조정
brightness_factors = np.arange(0.5, 1.4, 0.05)

for filename in os.listdir(input_image_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):
  
        image_path = os.path.join(input_image_dir, filename)
        image = cv2.imread(image_path)

        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(input_label_dir, label_filename)

        # 라벨 파일 복사
        for factor in brightness_factors:
            # 밝기 조정
            augmented_image = cv2.convertScaleAbs(image, alpha=factor, beta=0)

            
            output_image_filename = f"bright_{factor:.2f}_{filename}"
            output_image_path = os.path.join(output_image_dir, output_image_filename)
            cv2.imwrite(output_image_path, augmented_image)

           
            output_label_filename = f"bright_{factor:.2f}_{label_filename}"
            output_label_path = os.path.join(output_label_dir, output_label_filename)
            shutil.copy(label_path, output_label_path)

        print(f"{filename}의 밝기 조정 및 라벨 파일 복사 완료")

In [ ]:
#모델 학습
ultralytics.checks()

model = YOLO('yolov8x.pt')
print(type(model.names), len(model.names))

print(model.names)

model.train(data='./ultralytics/train_data/data.yaml', epochs=100)

In [ ]:
# 모델 로드
model = YOLO('./ultralytics/runs/detect/train4/weights/best.pt')

# 테스트 데이터셋을 이용하여 성능 평가
results = model.val(data='./ultralytics/train_data1/data.yaml', split='val')

# mAP와 IoU 확인
print(f"mAP@0.5: {results.box.map50:.4f}")
print(f"mAP@0.5:0.95: {results.box.map:.4f}")

In [ ]:
# 하이퍼 파라미터 튜닝

# 랜덤 서치를 위한 하이퍼파라미터 범위 설정
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [16, 32, 64]
epochs = [50, 100, 150]

# 여러 번 반복하면서 무작위 하이퍼파라미터로 모델 학습
best_map = 0
best_params = {}

for i in range(10):  # 10번의 랜덤 서치 수행
    lr = random.choice(learning_rates)
    batch = random.choice(batch_sizes)
    epoch = random.choice(epochs)
    
    print(f"Training with lr={lr}, batch={batch}, epochs={epoch}")
    
    model = YOLO('./ultralytics/runs/detect/train4/weights/best.pt')
    results = model.train(data='./ultralytics/train_data1/data.yaml', epochs=epoch, batch=batch, lr0=lr)
    
    # mAP 확인
    current_map = results.metrics.get('mAP50(B)', 0)
    
    if current_map > best_map:
        best_map = current_map
        best_params = {'lr': lr, 'batch': batch, 'epochs': epoch}

print(f"Best parameters: {best_params}, Best mAP: {best_map}")
